In [2]:
import pandas as pd

df = pd.read_csv("data/AI_Human.csv")
df = df.dropna()
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset

ds = load_dataset("Hello-SimpleAI/HC3", "all")

/home/sneha/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Extract and label data
rows = []
for entry in ds["train"]:
    for a in entry["chatgpt_answers"]:
        rows.append({"text": a.strip(), "label": 0})  # AI
    for a in entry["human_answers"]:
        rows.append({"text": a.strip(), "label": 1})  # Human

df = pd.DataFrame(rows).dropna()
print(f"Samples: {len(df)}")

Samples: 85449


In [6]:

# Balance classes
min_class = df['label'].value_counts().min()
df_balanced = pd.concat([
    df[df['label'] == 0].sample(min_class),
    df[df['label'] == 1].sample(min_class)
]).sample(frac=1).reset_index(drop=True)

In [7]:
# Tokenization + Dataset Prep
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

class HC3Dataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len)
        self.labels = labels

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = HC3Dataset(df_balanced['text'].tolist(), df_balanced['label'].tolist(), tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [8]:
# Train the RoBERTa Classifier
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=2e-5)

model.train()
for epoch in range(3):
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch + 1} Loss: {total_loss:.4f}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/sneha/venv/lib/python3.8/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 6726/6726 [16:55<00:00,  6.62it/s]


Epoch 1 Loss: 140.2325


100%|██████████| 6726/6726 [14:55<00:00,  7.51it/s]


Epoch 2 Loss: 56.9258


100%|██████████| 6726/6726 [14:54<00:00,  7.52it/s]

Epoch 3 Loss: 54.3202


In [10]:
!pip install jsonlines

In [12]:
!pip install scikit-learn

     |████████████████████████████████| 11.1 MB 31.3 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 103.2 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 94.4 MB/s eta 0:00:01


In [13]:
import os
import jsonlines
import torch
from sklearn.metrics import classification_report
from transformers import RobertaTokenizer
from tqdm import tqdm

dev_dir = "cs162-final-dev-main"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
human_texts, machine_texts = [], []

for filename in os.listdir(dev_dir):
    if filename.endswith(".jsonl"):
        file_path = os.path.join(dev_dir, filename)
        with jsonlines.open(file_path) as reader:
            for row in reader:
                human_texts.append(row["human_text"])
                machine_texts.append(row["machine_text"])

test_texts = human_texts + machine_texts
true_labels = [1] * len(human_texts) + [0] * len(machine_texts)

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Predict
model.eval()
preds = []

with torch.no_grad():
    for i in tqdm(range(0, len(test_texts), 8)):
        batch = test_texts[i:i+8]
        encodings = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=256)
        input_ids = encodings["input_ids"].to(device)
        attention_mask = encodings["attention_mask"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        batch_preds = torch.argmax(outputs.logits, dim=1).tolist()
        preds.extend(batch_preds)

# Evaluate
from sklearn.metrics import classification_report

print(classification_report(true_labels, preds, target_names=["AI-generated", "Human-written"]))

100%|██████████| 3000/3000 [02:11<00:00, 22.84it/s] 


               precision    recall  f1-score   support

 AI-generated       0.76      0.94      0.84     12000
Human-written       0.93      0.70      0.79     12000

     accuracy                           0.82     24000
    macro avg       0.84      0.82      0.82     24000
 weighted avg       0.84      0.82      0.82     24000



In [14]:
model.save_pretrained("initial-roberta-ai-detector")
tokenizer.save_pretrained("initial-roberta-ai-detector")

('initial-roberta-ai-detector/tokenizer_config.json',
 'initial-roberta-ai-detector/special_tokens_map.json',
 'initial-roberta-ai-detector/vocab.json',
 'initial-roberta-ai-detector/merges.txt',
 'initial-roberta-ai-detector/added_tokens.json')